In [ ]:
import numpy as np
import os
import random

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from keras import backend as K
from keras.models import Sequential ,Model
from keras.callbacks import ModelCheckpoint, History
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation,MaxPooling2D,UpSampling2D,Dense,BatchNormalization,Input,Reshape,multiply,add,Dropout,AveragePooling2D,GlobalAveragePooling2D,concatenate
from keras.layers.convolutional import Conv2D,Conv2DTranspose
from keras.regularizers import l2
from keras.engine import Layer,InputSpec
from keras.utils import conv_utils
from layers import BN_ReLU_Conv, TransitionDown, TransitionUp, SoftmaxLayer

In [ ]:
train_src_url = './dataset_liver/train_img'
train_mask_url = './dataset_liver/train_mask'

val_src_url = './dataset_liver/val_img'
val_mask_url = './dataset_liver/val_mask'

test_src_url = './dataset_liver/test_img'
test_mask_url = './dataset_liver/test_mask'

NO_OF_TRAINING_IMAGES = len(os.listdir('./dataset_liver/train_img/train'))
NO_OF_VAL_IMAGES = len(os.listdir('./dataset_liver/val_img/val'))
NO_OF_TEST_IMAGES = len(os.listdir('./dataset_liver/test_img/test'))


EPOCHS = 2
BATCH_SIZE = 2
img_h = 256
img_w = 256

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(  
        rescale=1.0 / 255.0,
        rotation_range=20,
        #width_shift_range=0.2,
        #height_shift_range=0.2,
    )

#train processing
train_generator = train_datagen.flow_from_directory(train_src_url,
                                                        target_size=(img_h, img_w),
                                                        batch_size=BATCH_SIZE,
                                                        color_mode='grayscale',
                                                        seed=123,
                                                        shuffle=True,
                                                        class_mode=None)
    
train_mask_generator = train_datagen.flow_from_directory(train_mask_url,
                                                        target_size=(img_h, img_w),
                                                        batch_size=BATCH_SIZE,
                                                        color_mode='grayscale',
                                                        seed=123,
                                                        shuffle=True,
                                                        class_mode=None)

#val processing
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1.0 / 255.0,
    )

val_generator = train_datagen.flow_from_directory(val_src_url,
                                                        target_size=(img_h, img_w),
                                                        batch_size=BATCH_SIZE,
                                                        color_mode='grayscale',
                                                        seed=1,
                                                        shuffle=True,
                                                        class_mode=None)
    
val_mask_generator = train_datagen.flow_from_directory(val_mask_url,
                                                        target_size=(img_h, img_w),
                                                        batch_size=BATCH_SIZE,
                                                        color_mode='grayscale',
                                                        seed=1,
                                                        shuffle=True,
                                                        class_mode=None)
#test processing
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1.0 / 255.0,
    )

test_generator = test_datagen.flow_from_directory(test_src_url,
                                                        target_size=(img_h, img_w),
                                                        batch_size=BATCH_SIZE,
                                                        color_mode='grayscale',
                                                        class_mode=None)
    
test_mask_generator = test_datagen.flow_from_directory(test_mask_url,
                                                        target_size=(img_h, img_w),
                                                        batch_size=BATCH_SIZE,  
                                                        color_mode='grayscale',
                                                        class_mode=None)
    
    
train_generator = zip(train_generator, train_mask_generator)
val_generator = zip(val_generator, val_mask_generator)
test_generator = zip(test_generator, test_mask_generator) 

In [ ]:
#https://github.com/mad-Ye/FC-DenseNet-Keras
def Tiramisu(
        input_shape=(256,256,1),
        n_classes = 3,
        n_filters_first_conv = 48,
        n_pool = 5,
        growth_rate = 16 ,
        n_layers_per_block = [4,5,7,10,12,15,12,10,7,5,4],
        dropout_p = 0
        ):
    if type(n_layers_per_block) == list:
            print(len(n_layers_per_block))
    elif type(n_layers_per_block) == int:
            n_layers_per_block = [n_layers_per_block] * (2 * n_pool + 1)
    else:
        raise ValueError
        
#####################
# First Convolution #
#####################        
    inputs = Input(shape=input_shape)
    stack = Conv2D(filters=n_filters_first_conv, kernel_size=3, padding='same', kernel_initializer='he_uniform')(inputs)
    n_filters = n_filters_first_conv

#####################
# Downsampling path #
#####################     
    skip_connection_list = []
    
    for i in range(n_pool):
        for j in range(n_layers_per_block[i]):
            l = BN_ReLU_Conv(stack, growth_rate, dropout_p=dropout_p)
            stack = concatenate([stack, l])
            n_filters += growth_rate
        
        skip_connection_list.append(stack)        
        stack = TransitionDown(stack, n_filters, dropout_p)
    skip_connection_list = skip_connection_list[::-1]

    
#####################
#    Bottleneck     #
#####################     
    block_to_upsample=[]
    
    for j in range(n_layers_per_block[n_pool]):
        l = BN_ReLU_Conv(stack, growth_rate, dropout_p=dropout_p)
        block_to_upsample.append(l)
        stack = concatenate([stack,l])
    block_to_upsample = concatenate(block_to_upsample)

#####################
#  Upsampling path  #
#####################
    for i in range(n_pool):
        n_filters_keep = growth_rate * n_layers_per_block[n_pool + i ]
        stack = TransitionUp(skip_connection_list[i], block_to_upsample, n_filters_keep)
        
        block_to_upsample = []
        for j in range(n_layers_per_block[ n_pool + i + 1 ]):
            l = BN_ReLU_Conv(stack, growth_rate, dropout_p=dropout_p)
            block_to_upsample.append(l)
            stack = concatenate([stack, l])
        block_to_upsample = concatenate(block_to_upsample)

#####################
#  Softmax          #
#####################
    output = SoftmaxLayer(stack, 1)            
    model=Model(inputs = inputs, outputs = output)    
    model.summary()
    
    return model

In [ ]:
train = Tiramisu()

In [ ]:
# dice_loss

def dice_coef(y_true, y_pred):
    smooth = 1.
    # Flatten
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coef(y_true, y_pred)
    return loss

In [ ]:
# learning rate decay 

from keras.callbacks import LearningRateScheduler
lr_base = 1e-5
lr_power = 0.995

def lr_scheduler(epoch):
    lr = (float(lr_base) ** float(lr_power)) ** float(epoch + 1)  
    return lr

lrate = LearningRateScheduler(lr_scheduler)

In [ ]:
train.compile(optimizer=Adam(lr=lr_base, beta_1=0.9, beta_2=0.999) , loss=dice_loss, metrics=[dice_coef])
#train.compile(optimizer=Lookahead(Adam(lr_base)), loss=dice_loss, metrics=[dice_coef])
#train.compile(optimizer=RMSprop(lr=lr_base), loss=dice_loss, metrics=[dice_coef])
#train.compile(optimizer=RMSprop(lr=lr_base), loss=dice_p_bce, metrics=[dice_coef])
model_checkpoint = ModelCheckpoint('denseunet_weight_liver_test.h5', monitor='val_loss', save_best_only=True)
history =  train.fit_generator(train_generator, epochs=EPOCHS, steps_per_epoch = NO_OF_TRAINING_IMAGES//BATCH_SIZE, 
                               validation_data=val_generator, validation_steps= NO_OF_VAL_IMAGES//BATCH_SIZE, 
                               verbose=1, shuffle=True,
                               callbacks=[model_checkpoint, lrate])

In [ ]:
#Plot train processing 

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure(figsize=(7,10),dpi=100)
plt.subplot(2,1,1)
plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('model acc')
plt.ylabel('dice_coef')
plt.xlabel('epoch')
plt.legend(['train','test'],loc = 'upper right')

In [ ]:
plt.figure(figsize=(7,10),dpi=100)
plt.subplot(2,1,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'],loc = 'upper right')

In [ ]:
#save model

In [ ]:
train.save_weights("denseunet_weight_liver_final_test.h5")

In [ ]:
test = Tiramisu()
test.load_weights("denseunet_weight_liver_final_test.h5")

In [ ]:
#predict

In [ ]:
from skimage.transform import resize
import skimage.io as io

In [ ]:
coll = io.ImageCollection('./seg_liver_result/*.png')
test_scans = io.concatenate_images(coll)
test_scans = test_scans/ 255.0
test_scans = resize(test_scans, (test_scans.shape[0], img_h, img_w, 1))
test_scans = np.reshape(test_scans ,(-1, img_h, img_w,1))

In [ ]:
imgs_mask_test = test.predict(test_scans, verbose=1)

In [ ]:
for j in range(3):
        print ('scan '+str(j))
        f, ax = plt.subplots(1, 3,figsize=(10,5))
        ax[0].imshow(test_scans[j,:,:,0],cmap=plt.cm.gray)
        imgs_mask_test[imgs_mask_test > 0.5] = 1
        imgs_mask_test[imgs_mask_test < 0.5] = 0
        ax[1].imshow((imgs_mask_test[j,:,:,0] * 255.).astype(np.uint8) ,cmap=plt.cm.gray)
        ax[2].imshow((test_scans[j,:,:,0])*(imgs_mask_test[j,:, :, 0]) ,cmap=plt.cm.gray)
        plt.show()

In [ ]:
import cv2
i = 0
for mask,scans in zip(imgs_mask_test,test_scans) :
    i+=1
    mask[mask > 0.5] = 1
    mask[mask < 0.5] = 0
    image = scans[:,:,0]*mask[:,:,0]
    pic = cv2.resize(image, (512, 512), interpolation=cv2.INTER_CUBIC)
    io.imsave(os.path.join(r'./seg_liver_result', str(i) + '_pred.png'), pic)
    

In [ ]:
#evaluate

In [ ]:
test.compile(optimizer=Adam(lr=1e-5, beta_1=0.9, beta_2=0.999) , loss=dice_loss, metrics=[dice_coef])

In [ ]:
acc, loss = test.evaluate_generator(test_generator, steps=NO_OF_TEST_IMAGES//BATCH_SIZE+1 ,verbose=1)

In [ ]:
acc

In [ ]:
loss